# Build `uni_gram_freq`

In [1]:
import os
import threading
from utils.tdmstudio import get_title_and_text
from utils.tokenizer import Tokenizer
from tqdm import tqdm
tokenizer = Tokenizer()

data_sources = ['/home/ec2-user/SageMaker/data/GM_all_1945_1956/']
files = [data_source+file for data_source in data_sources for file in os.listdir(data_source)]

freq = {}
count=1
for token_set in tqdm(map(lambda file: set(tokenizer.tokenize(get_title_and_text(file))), files)):
    for token in token_set:
        if not token in freq:
            freq[token]=1
        else:
            freq[token]+=1
    if count%200000==0:
        freq = dict([(term,freq[term]) for term in list(filter(lambda term: freq[term]>1 ,freq ))])
    if count==2000:
        break
    count+=1
    
sorted_list = sorted([(word,freq[word]) for word in freq], key=lambda x: x[1],reverse=True)

freq_filename = 'precomputed/uni_gram_freq.txt'
with open(freq_filename,  'w') as w:
    w.write('\n'.join([f'{word};{count}' for word, count in sorted_list[:20000]]))
    


1999it [06:28,  5.14it/s]


In [2]:
!head precomputed/uni_gram_freq.txt

title;777
new;711
year;675
toronto;670
st;568
say;551
day;541
time;536
today;527
ad;525


10000

# Build `bi_gram_freq`

In [4]:

list_=['The','quick','brown','lazy','fox']
list(ngrams(list_))

[('The', 'quick'),
 ('quick', 'brown'),
 ('brown', 'lazy'),
 ('lazy', 'fox'),
 ('The', 'quick', 'brown'),
 ('quick', 'brown', 'lazy'),
 ('brown', 'lazy', 'fox')]

In [11]:
import os
import threading
from utils.tdmstudio import get_title_and_text
from utils.tokenizer import Tokenizer
from tqdm import tqdm

def _bigrams(list_):
    return list(zip(list_[:-1], list_[1:]))
def _trigrams(list_):
    return list(zip(list_[:-2], list_[1:-1], list_[2:]))
def ngrams(list_):
    list_=list(list_)
    return _bigrams(list_)+_trigrams(list_)

tokenizer = Tokenizer()

data_sources = ['/home/ec2-user/SageMaker/data/GM_all_1945_1956/']
files = [data_source+file for data_source in data_sources for file in os.listdir(data_source)]

unigram_freq = ([(line.split(';')[0], int(line.split(';')[1])) 
             for line in open('precomputed/uni_gram_freq.txt','r').read().splitlines()])
unigram_freq = sorted(unigram_freq, key=lambda x: x[1], reverse=True)[:10000]
unigram_freq = dict(unigram_freq)
len(unigram_freq)


ngram_freq = {}
count=1
for token_list in tqdm(map(lambda file: tokenizer.tokenize(get_title_and_text(file)), files)):
    for ngram in ngrams(filter(lambda token: token in unigram_freq , token_list)):
        if not ngram in ngram_freq:
            ngram_freq[ngram]=1
        else:
            ngram_freq[ngram]+=1
    if count%200==0: # 1% (0.01)
        ngram_freq = dict([(ngram,ngram_freq[ngram]) for ngram in list(filter(lambda ngram: ngram_freq[ngram]>1,ngram_freq))])        
    if count==3000:
        break
    count+=1
    
sorted_list = sorted([(word,ngram_freq[word]) for word in ngram_freq], key=lambda x: x[1],reverse=True)

freq_filename = 'precomputed/ngram_freq.txt'
with open(freq_filename,  'w') as w:
    w.write('\n'.join([f'{word};{count}' for word, count in sorted_list[:20000]]))


2999it [01:32, 32.46it/s]


# Precompute BoW vector (uni-, bi- and tri-gram) (MATRIX)

In [8]:
from scipy import sparse
from sklearn.preprocessing import normalize
import os
from tqdm import tqdm
from utils.tdmstudio import get_title_and_text
from utils.tokenizer import Tokenizer

tokenizer = Tokenizer()

freq = [(line.split(';')[0], line.split(';')[1]) for line in open('precomputed/uni_gram_freq.txt').read().splitlines()]
freq += [(line.split(';')[0], line.split(';')[1]) for line in open('precomputed/ngram_freq.txt').read().splitlines()]
freq = sorted(freq, key=lambda x: x[1],reverse=True)[:10000]
freq=dict(freq)
vocab = list([word for word in freq])
word2index = dict([(word,idx) for idx,word in enumerate(vocab)])

with open('precomputed/vocab.txt','w') as f:
    f.write('\n'.join(vocab))

data_sources = ['/home/ec2-user/SageMaker/data/GM_all_1945_1956/']
files = [data_source+file for data_source in data_sources for file in os.listdir(data_source)][:100]

with open('precomputed/columns.txt','w') as f:
    f.write('\n'.join(map(lambda x: x.split('/')[-1], files)))

m = sparse.csc_matrix((len(files), len(vocab)))


count = 0
for token_list in tqdm(map(lambda file: tokenizer.tokenize(get_title_and_text(file)), files)):
    ngram_list = list(token_list)
    ngram_list += [' '.join(ngram) for ngram in Tokenizer.ngrams(ngram_list)]
        
    ngram_list = filter(lambda ngram: ngram in word2index , ngram_list)
    for ngram in ngram_list:
        m[count, word2index[ngram]]+=1
    count+=1

m = normalize(m, axis=1)


sparse.save_npz("yourmatrix.npz", m)

0it [00:00, ?it/s]/home/ec2-user/SageMaker/.conda/envs/imm/lib/python3.9/site-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
100it [00:06, 15.10it/s]


In [85]:
import sys
sys.path.append('..')

import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
import os
from tqdm import tqdm
from utils.tdmstudio import get_title_and_text
from utils.tokenizer import Tokenizer
import re
import pickle

import concurrent.futures

from threading import Lock
wlock = Lock()

# if __name__=='__main__':
tokenizer = Tokenizer()

input_ =  ' --output-file=precomputed/tfidf_matrix.npz /home/ec2-user/SageMaker/data/GM_all_1945_1956/ '\
' --vocab-file=precomputed/vocab.txt --idf-file=precomputed/idf.txt --debug' 

# INPUT #1
output_file = re.findall('--output-file=([^\ ]*)', input_)
if len(output_file)==0:
    print('Please indicate output file (python script.py --output-file=../precomputed/tfidf_matrix.npz')
    sys.exit(1)
output_file = output_file[0]

# INPUT #2
data_sources = [arg for arg in input_.split(' ') if '--output-file' not in arg]
data_sources = [data_source for data_source in data_sources if os.path.exists(data_source)]
if len(data_sources)==0:
    print('Please provide data sources (/home/ec2-user/SageMaker/data/GM_all_1945_1956/)')
    sys.exit(1)

# INPUT #3 
debug = False
debug = len(re.findall('--debug', input_))>0


# INPUT #4
vocab_file = re.findall('--vocab-file=([^\ ]*)', input_)
if len(vocab_file)==0:
    print('Please indicate input file (python script.py --vocab-file=../precomputed/vocab.txt')
    sys.exit(1)
vocab_file = vocab_file[0]

# INPUT #5
idf_file = re.findall('--idf-file=([^\ ]*)', input_)
if len(idf_file)==0:
    print('Please indicate input file (python script.py --idf-file=../precomputed/idf.txt')
    sys.exit(1)
idf_file = idf_file[0]

# # INPUT #6
# column_file = re.findall('--columns-file=([^\ ]*)', input_)
# if len(column_file)==0:
#     print('Please indicate output file (python script.py --columns-file=../precomputed/columns.txt')
#     sys.exit(1)
# column_file = column_file[0]


###        
vocab = open(vocab_file, 'r').read().splitlines()
word2index = dict([(word,idx) for idx,word in enumerate(vocab)])


files = [data_source+file for data_source in data_sources for file in os.listdir(data_source)]
if debug:
    print('[WARNING] debug activated.')
    files = files[:10]

# with open(column_file, 'w') as f:
#     f.write('\n'.join(map(lambda x: x.split('/')[-1], files)))

m = sparse.lil_matrix((len(files), len(vocab)))
idf = sparse.lil_matrix(np.array([float(value) for value in open(idf_file, 'r').read().splitlines()]).reshape(1,-1))

print('Starting...')
writer = open('computing_tfidf_all_corpus.txt','a')

def process_file(file):
    precomputed_file = f"/home/ec2-user/SageMaker/mariano/notebooks/04. Model of DP/precomputed/{file.split('/')[-1][:-4]}"
    precomputed_file = precomputed_file+'_glove.p'
    if os.path.isfile(precomputed_file):
        list_ = pickle.load(open(precomputed_file,'rb'))
        if len(list_)==4:
            vec = sparse.lil_matrix((1, len(vocab)))
            token_list = tokenizer.tokenize(get_title_and_text(file))
            ngram_list = list(token_list)
            ngram_list += [' '.join(ngram) for ngram in Tokenizer.ngrams(ngram_list)]

            ngram_list = filter(lambda ngram: ngram in word2index , ngram_list)
            for ngram in ngram_list:
                vec[0, word2index[ngram]]+=1
            vec = vec.multiply(idf)
            vec = normalize(vec, axis=1)
            list_.append(vec)
            assert len(list_)==5
            assert list_[0].shape==(300,) and list_[1].shape==(600,) 
            assert list_[2].shape==(1,10000) and list_[3].shape==(1,10000) and list_[4].shape==(1,10000)
            assert type(list_[0])==np.ndarray and type(list_[1])==np.ndarray
            assert type(list_[2])==sparse._csr.csr_matrix
            assert type(list_[3])==sparse._csr.csr_matrix
            assert type(list_[4])==sparse._csr.csr_matrix
            pickle.dump(list_, open(precomputed_file, 'wb'))
            wlock.acquire()
            writer.write(f'{id_}\n')
            writer.flush()
            wlock.release()
            
            


with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(process_file, files, chunksize=4000)
writer.close()


[WARNING] debug activated.
Starting


/tmp/ipykernel_12272/261984976.py:85: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  list_ = pickle.load(open(precomputed_file,'rb'))


KeyboardInterrupt: 

In [90]:
list_[-1].count_nonzero()

95

In [92]:
for file in files[:10]:
    precomputed_file = f"/home/ec2-user/SageMaker/mariano/notebooks/04. Model of DP/precomputed/{file.split('/')[-1][:-4]}"
    precomputed_file = precomputed_file+'_glove.p'
    precomputed_file
    list_ = pickle.load(open(precomputed_file, 'rb'))
    print(f'{len(list_)}'+','.join(str(vec.shape) for vec in list_)+ ','
          + ','.join([str(vec.count_nonzero()) for vec in list_ if not type(vec)==np.ndarray]))
#     print(len(pickle.load(open(precomputed_file, 'rb'))))

5(300,),(600,),(1, 10000),(1, 10000),(1, 10000),225,225,248
5(300,),(600,),(1, 10000),(1, 10000),(1, 10000),29,29,32
5(300,),(600,),(1, 10000),(1, 10000),(1, 10000),8,8,12
5(300,),(600,),(1, 10000),(1, 10000),(1, 10000),52,52,62
4(300,),(600,),(1, 10000),(1, 10000),61,61
4(300,),(600,),(1, 10000),(1, 10000),233,233
4(300,),(600,),(1, 10000),(1, 10000),114,114
4(300,),(600,),(1, 10000),(1, 10000),14,14
4(300,),(600,),(1, 10000),(1, 10000),72,72
4(300,),(600,),(1, 10000),(1, 10000),95,95


/tmp/ipykernel_12272/2166003331.py:5: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  list_ = pickle.load(open(precomputed_file, 'rb'))


In [ ]:
%%time 


In [18]:
list_ = process_file(files[0])
print(len(list_))
for idx,elem in enumerate(list_):
    print(f'{idx}, {type(elem)}, {elem.shape}')
#     print(type(elem))
list_[2:]

5
0, <class 'numpy.ndarray'>, (300,)
1, <class 'numpy.ndarray'>, (600,)
2, <class 'scipy.sparse._csr.csr_matrix'>, (1, 10000)
3, <class 'scipy.sparse._csr.csr_matrix'>, (1, 10000)
4, <class 'scipy.sparse._csr.csr_matrix'>, (1, 10000)


/tmp/ipykernel_25543/1820842794.py:83: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  list_ = pickle.load(open(precomputed_file,'rb'))


[<1x10000 sparse matrix of type '<class 'numpy.int32'>'
 	with 225 stored elements in Compressed Sparse Row format>,
 <1x10000 sparse matrix of type '<class 'numpy.int32'>'
 	with 225 stored elements in Compressed Sparse Row format>,
 <1x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 248 stored elements in Compressed Sparse Row format>]

In [12]:
import pickle
errors=0
for file in files:


    if not os.path.isfile(precomputed_file):
        errors+=1
errors

/tmp/ipykernel_25543/3013387156.py:6: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  list_ = pickle.load(open(precomputed_file,'rb'))


0

In [3]:
print(data_sources)

['/home/ec2-user/.local/share/jupyter/runtime/kernel-deb2ef83-8d6f-45f6-ad34-537ce8264bcb.json']


# IDF

In [33]:
import os 
from tqdm import tqdm
from utils.tdmstudio import get_title_and_text
from utils.tokenizer import Tokenizer
import numpy as np

tokenizer = Tokenizer()

unigram_file = 'precomputed/uni_gram_freq.txt'
ngram_file = 'precomputed/ngram_freq.txt'
freq = [(line.split(';')[0], int(line.split(';')[1])) for line in open(unigram_file).read().splitlines()]
freq += [(line.split(';')[0], int(line.split(';')[1])) for line in open(ngram_file).read().splitlines()]

freq = sorted(freq, key=lambda x: x[1],reverse=True)[:10000]

freq=dict(freq)
vocab = list([word for word in freq])

len(vocab)

data_sources = ['/home/ec2-user/SageMaker/data/GM_all_1945_1956/']
files = [data_source+file for data_source in data_sources for file in os.listdir(data_source)][:1]
    
freq = dict([(word,0) for word in vocab])
for token_list in tqdm(map(lambda file: tokenizer.tokenize(get_title_and_text(file)), files)):
    ngram_list = list(token_list)
    ngram_list += [' '.join(ngram) for ngram in Tokenizer.ngrams(ngram_list)]
        
    ngram_list = filter(lambda ngram: ngram in freq , ngram_list)
    
    for ngram in set(ngram_list):
        freq[ngram]+=1
        
N=len(files)
nidf=np.array([freq[word] for word in vocab])
idf = np.log(N/(1+np.array([freq[word] for word in vocab])))

with open('precomputed/idf.txt','w') as f:
    f.write('\n'.join([str(value) for value in idf]))



1it [00:00, 13.87it/s]


In [34]:
print(get_title_and_text(files[0]))

Today and Tomorrow.
          
            

 

Today and Tomorrow


Walter Lippmann's


The Voice of America


Though money for radio broadcasts and printing is needed so that the Voice of America may be heard in foreign lands our greatest need is to have something definite clear and convincing for that voice to say There will he little opposition in Congress to an appropriation if


it were not for the feeling that the men who conduct our propaganda have little to do with the making of our policy and that the sales department of the Government so lo speak writing about goods for which the produc tion engineers have just begun to make the first blueprints As for the customers abroad they are undoubtedly confused and suspicious partly no doubt because the rival firm misrepresents us but chiefly because we sound hot and bothered when as Great Power they expect us to be cool and definite Mr Benton's difficul ties with Congress and with the opposition abroad will diminish when his chief S

In [35]:
' --- '.join(np.array(vocab)[nidf!=0])

'year --- new --- mr --- time --- man --- ad --- today --- ing --- good --- work --- home --- end --- tion --- give --- way --- long --- united --- ed --- de --- ment --- lo --- world --- great --- point --- large --- government --- er --- sale --- follow --- need --- little --- far --- east --- call --- hand --- state --- war --- port --- pro --- return --- british --- well --- land --- power --- think --- states --- department --- american --- oil --- red --- force --- money --- send --- provide --- form --- write --- able --- united states --- stand --- term --- centre --- western --- expect --- tie --- condition --- tho --- begin --- union --- program --- require --- firm --- believe --- hear --- chief --- cover --- secretary --- radio --- french --- hard --- speak --- kind --- strike --- clear --- support --- mill --- sound --- credit --- policy --- hot --- basis --- enter --- nation --- ami --- britain --- division --- standard --- necessary --- capital --- foreign --- tomorrow -

In [87]:
from sklearn.preprocessing import normalize
m = normalize(m, axis=1)

In [90]:
import numpy as np
print(np.average(m.toarray()[:100,:],axis=0).shape)
np.linalg.norm(m.toarray()[:100,:],axis=0)

(10000,)


array([0.        , 0.        , 0.70710678, ..., 0.        , 0.12803688,
       0.        ])

In [9]:
import numpy as np
print(np.average(m.toarray()[:100,:],axis=1).shape)
np.linalg.norm(m.toarray()[:100,:],axis=1)

(100,)


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [67]:
m

<10000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 330901 stored elements in Compressed Sparse Column format>

In [ ]:
# 10000it [33:13,  5.02it/s]
# <10000x10000 sparse matrix of type '<class 'numpy.float64'>'
#	with 330901 stored elements in Compressed Sparse Column format>

In [58]:
m

<10000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Column format>

In [29]:
cant_doc=2000000
no_features=len(vocab)


m[0,0]=3
m

/home/ec2-user/SageMaker/.conda/envs/imm/lib/python3.9/site-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


<2000000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Column format>

In [10]:
sorted_list

[(('ad', 'title'), 603),
 (('new', 'york'), 558),
 (('display', 'ad'), 544),
 (('display', 'ad', 'title'), 543),
 (('mr', 'mrs'), 519),
 (('globe', 'mail'), 373),
 (('united', 'states'), 359),
 (('st', 'toronto'), 293),
 (('box', 'globe'), 252),
 (('year', 'old'), 247),
 (('co', 'ltd'), 193),
 (('dining', 'room'), 187),
 (('living', 'room'), 182),
 (('year', 'ago'), 180),
 (('real', 'estate'), 168),
 (('yonge', 'st'), 167),
 (('ft', 'ft'), 164),
 (('ave', 'toronto'), 160),
 (('st', 'clair'), 154),
 (('day', 'week'), 146),
 (('funeral', 'home'), 146),
 (('globe', 'mall'), 142),
 (('hot', 'water'), 135),
 (('high', 'school'), 127),
 (('mrs', 'mrs'), 127),
 (('box', 'globe', 'mail'), 126),
 (('high', 'low'), 118),
 (('solid', 'brick'), 104),
 (('toronto', 'phone'), 102),
 (('clock', 'interment'), 99),
 (('son', 'mr'), 92),
 (('mr', 'say'), 88),
 (('prime', 'minister'), 87),
 (('general', 'hospital'), 86),
 (('public', 'school'), 84),
 (('bloor', 'st'), 83),
 (('ft', 'ft', 'ft'), 83),
 (('

In [9]:
ngram_freq

{('today', 'tomorrow'): 5,
 ('voice', 'america'): 3,
 ('pro', 'gram'): 16,
 ('american', 'conflict'): 2,
 ('conflict', 'far'): 2,
 ('european', 'guaranty'): 2,
 ('great', 'britain'): 25,
 ('pos', 'sible'): 11,
 ('european', 'economic'): 3,
 ('economic', 'union'): 4,
 ('lo', 'provide'): 4,
 ('guaranty', 'german'): 2,
 ('western', 'europe'): 10,
 ('united', 'states'): 359,
 ('military', 'evacuation'): 2,
 ('soviet', 'union'): 40,
 ('union', 'united'): 2,
 ('war', 'time'): 9,
 ('gov', 'ernment'): 29,
 ('european', 'economic', 'union'): 3,
 ('court', 'room'): 21,
 ('regina', 'reglna'): 2,
 ('reglna', 'regina'): 2,
 ('guaranty', 'trust'): 8,
 ('con', 'struction'): 12,
 ('co', 'ltd'): 193,
 ('ltd', 'johnson'): 3,
 ('display', 'ad'): 544,
 ('ad', 'title'): 603,
 ('display', 'ad', 'title'): 543,
 ('run', 'play'): 3,
 ('1h', 'yard'): 2,
 ('easy', 'use'): 4,
 ('com', 'plete'): 20,
 ('pattern', 'dept'): 7,
 ('dept', 'globe'): 8,
 ('globe', 'mall'): 142,
 ('st', 'toronto'): 293,
 ('toronto', 'ont'

In [7]:
len(freq)

13463

In [8]:
tokens =

In [9]:
next(tokens)

{'ability',
 'able',
 'abroad',
 'actual',
 'ad',
 'advance',
 'afford',
 'agreement',
 'alter',
 'ameri',
 'america',
 'american',
 'ami',
 'apparent',
 'appropriation',
 'ard',
 'armle',
 'army',
 'artificial',
 'assistance',
 'ation',
 'attain',
 'attainable',
 'attraction',
 'balance',
 'basis',
 'bate',
 'begin',
 'believe',
 'benton',
 'blueprint',
 'border',
 'bother',
 'britain',
 'british',
 'broadcast',
 'call',
 'capital',
 'centre',
 'chief',
 'chiefly',
 'civilized',
 'clear',
 'coali',
 'compel',
 'comprehend',
 'concession',
 'concrete',
 'condition',
 'conduct',
 'conflict',
 'confused',
 'congress',
 'consciously',
 'consideration',
 'continent',
 'contribute',
 'contribution',
 'convincing',
 'cool',
 'cover',
 'credit',
 'customer',
 'dardanelles',
 'de',
 'deep',
 'definite',
 'department',
 'difficul',
 'diminish',
 'discussion',
 'divide',
 'division',
 'doctrine',
 'doubt',
 'east',
 'eastern',
 'economic',
 'ed',
 'element',
 'emerge',
 'enable',
 'end',
 'engin

In [6]:
print(tokenizer.tokenize(get_title_and_text(files[0])))

['today', 'tomorrow', 'today', 'tomorrow', 'walter', 'lippmann', 'voice', 'america', 'money', 'radio', 'broadcast', 'printing', 'need', 'voice', 'america', 'hear', 'foreign', 'land', 'great', 'need', 'definite', 'clear', 'convincing', 'voice', 'little', 'opposition', 'congress', 'appropriation', 'feeling', 'man', 'conduct', 'propaganda', 'little', 'making', 'policy', 'sale', 'department', 'government', 'lo', 'speak', 'write', 'good', 'produc', 'tion', 'engineer', 'begin', 'blueprint', 'customer', 'abroad', 'undoubtedly', 'confused', 'suspicious', 'partly', 'doubt', 'rival', 'firm', 'misrepresent', 'chiefly', 'sound', 'hot', 'bother', 'great', 'power', 'expect', 'cool', 'definite', 'mr', 'benton', 'difficul', 'tie', 'congress', 'opposition', 'abroad', 'diminish', 'chief', 'secretary', 'marshall', 'time', 'form', 'concrete', 'american', 'pro', 'gram', 'settlement', 'war', 'program', 'settlement', 'element', 'program', 'believe', 'gradually', 'emerge', 'vast', 'ami', 'intricate', 'de', 'b

In [12]:
files[0]

'/home/ec2-user/SageMaker/data/GM_all_1945_1956/1323614655.xml'